In [24]:
import os
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [25]:
#edit sesuai kebutuhan
vessel_type = 'bulk carrier'
filenum = '1'
kelas = 'DNV'

In [26]:
#directory file csv
read_path = 'C:/Users/abinp/Documents/ITS/side projects/new/ship data scrapping/input/{}/split_{}.csv'.format(vessel_type,filenum)
write_path = 'C:/Users/abinp/Documents/ITS/side projects/new/ship data scrapping/output/{}/{}/{}.csv'.format(kelas,vessel_type,filenum)

In [27]:
# Check if input file exists
if os.path.exists(read_path):
    print(f"CSV read file '{read_path}' is found, continuing...")
else:
    print(f"CSV read file '{read_path}' is not found")

# Check if output file exists and create it if necessary
if os.path.exists(write_path):
    print(f"CSV write file '{write_path}' is found, continuing...")
else:
    print(f"CSV write file '{write_path}' is not found, creating one...")
    os.makedirs(os.path.dirname(write_path), exist_ok=True) # Create output directory if it doesn't exist
    with open(write_path, 'w', newline='') as f:
        writer = csv.writer(f)

CSV read file 'C:/Users/abinp/Documents/ITS/side projects/new/ship data scrapping/input/bulk carrier/split_1.csv' is found, continuing...
CSV write file 'C:/Users/abinp/Documents/ITS/side projects/new/ship data scrapping/output/bulk carrier/1.csv' is not found, creating one...


In [28]:
driver = webdriver.Chrome(executable_path=r"chromedriver")
driver.get("https://vesselregister.dnv.com/vesselregister/")

In [37]:
# Set up the webdriver
driver = webdriver.Chrome()

# Define the fieldnames for the CSV file
fieldnames = ['Name', 'Loa', 'Lbp', 'Lload', 'Bext', 'B', 'D', 'Draught', 'GT (ITC 69)', 'NT (ITC 69)', 'DWT', 'GT (PRE 69)', 'Freeboard']

# Open the CSV file for writing
with open(write_path, mode='w', newline='') as details_file:
    writer = csv.DictWriter(details_file, fieldnames=fieldnames)
    writer.writeheader()

    # Open the input file and loop through each vessel name
    with open(read_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # skip the first row (header)
        for row in reader:
            vessel_name = row[0]
            driver.get('https://vesselregister.dnv.com/vesselregister/')

            # Input the vessel name into the search field and press enter
            search_field = driver.find_element(By.CLASS_NAME, 'ant-input.search-field')
            search_field.send_keys(vessel_name)
            search_field.send_keys(Keys.RETURN)

            # Wait for the search results to load and click on the first vessel
            time.sleep(2)  # adjust this if necessary
            try:
                first_vessel = driver.find_element(By.XPATH, '/html/body/div[1]/section/main/div/div[3]/div[2]/div/div/div/div/div/table/tbody/tr[2]/td[1]/a')
                first_vessel.click()
                csvfile = open(read_path, 'r') # Re-open the input file in read mode
                lines = csvfile.readlines() # Read all the lines in the file
                csvfile.close()
                csvfile = open(read_path, 'w') # Re-open the input file in write mode
                csvfile.writelines(lines[1:]) # Write all the lines except the first line
                csvfile.close()
                print(f"vessel found on IMO : {vessel_name}")

            except:
                continue
        
            # Click the collapse/expand button to show the vessel size/dimensions
            time.sleep(5)
            dimensions_btn = driver.find_element(By.XPATH, '/html/body/div/section/main/div/div[2]/div[8]')
            dimensions_btn.click()
        
            # Scrape the vessel information and store it in a dictionary
            time.sleep(5)  # adjust this if necessary
            vessel_info = driver.find_element(By.XPATH, '/html/body/div/section/main/div/div[2]/div[8]/div[2]/div/div/div').text
            vessel_name = driver.find_element(By.XPATH, '/html/body/div/section/main/div/div[1]/div[2]/div[1]/span[1]').text
            details_dict = {'Name': vessel_name}
            dimension_values = driver.find_elements(By.XPATH, '/html/body/div/section/main/div/div[2]/div[8]/div[2]/div/div/div/div')
            for dim_value in dimension_values:
                if ':' not in dim_value.text:
                    continue
                dim_name, dim_val = dim_value.text.split(':', maxsplit=1)
                dim_name = dim_name.strip()
                dim_val = dim_val.strip()
                if dim_name in fieldnames:
                    details_dict[dim_name] = dim_val


            # Write the details to the CSV file
            writer.writerow(details_dict)

# Close the webdriver
driver.quit()


vessel found on IMO : 9486300
vessel found on IMO : 9468619
vessel found on IMO : 9524815
